In [1]:
import networkx as nx
import math

In [25]:
#读取
G1 = nx.read_edgelist('seed_G1.edgelist')
G2 = nx.read_edgelist('seed_G2.edgelist')
with open('seed_node_pairs.txt','r') as sp:
    lines = sp.readlines()

In [26]:
#获得G1 匹配的  与 G2 匹配的 并存入字典
g1p = []
g1pdir = {}
g2p =[]
g2pdir = {}

In [27]:
for line in lines:
    line = line.strip('\n')
    line = line.split(' ')
    g1p.append(line[0])
    g2p.append(line[1])
    g1pdir[line[0]] = line[1]
    g2pdir[line[1]] = line[0]

In [29]:
#获得未匹配过的 G1 与 G2 的 点
l = []
for i in G1.nodes:
    l.append(int(i))
y = sorted(l)
with open('G1.txt','w') as f:
    for j in range(len(y)):
        f.write(str(y[j])+ "\n") 
